In [1]:
from tensorflow.keras import backend as K  # type: ignore
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # delete alert message from Tensorflow
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array  # type: ignore
import numpy as np

2024-04-14 17:02:31.320489: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_pred = tf.argmax(y_pred, axis=-1)

    y_true = K.cast(y_true, dtype=K.floatx())
    y_pred = K.cast(y_pred, dtype=K.floatx())

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f)
                                           + K.sum(y_pred_f) + smooth)

In [3]:
y_true_path = "../Data/masks/aachen_000000_000019.png"
y_pred_path = "../Data/masks/aachen_000000_000019.png"

In [4]:
input_shape2 = (256, 256, 3)

def imgToArray(path):
    mask = path
    mask = img_to_array(Image.open(y_true_path).resize(input_shape2[:2], Image.NEAREST))
    mask = mask.astype(np.uint8)
    for x in range(mask.shape[0]):
        for y in range(mask.shape[1]):
            if (mask[x, y] == [250, 170, 30]).all():
                mask[x, y] = 0
            elif (mask[x, y] == [0, 0, 142]).all():
                mask[x, y] = 1
            elif (mask[x, y] == [102, 102, 156]).all():
                mask[x, y] = 2
            elif (mask[x, y] == [220, 20, 60]).all():
                mask[x, y] = 3
            elif (mask[x, y] == [153, 153, 153]).all():
                mask[x, y] = 4
            elif (mask[x, y] == [244, 35, 232]).all():
                mask[x, y] = 5
            elif (mask[x, y] == [70, 70, 70]).all():
                mask[x, y] = 6
            elif (mask[x, y] == [70, 130, 180]).all():
                mask[x, y] = 7
    mask = mask[:, :, 0:1]
    return mask

In [5]:
y_pred = imgToArray(y_true_path)
y_true = imgToArray(y_pred_path)

In [6]:
y_pred.shape

(256, 256, 1)

In [67]:
test1 = dice_coef(y_true, y_pred)

In [68]:
print(test1)

tf.Tensor(6.154755e-12, shape=(), dtype=float32)


In [43]:
import tensorflow.keras.backend as K

def DiceLoss(y_true, y_pred, smooth=1e-6):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    
    inputs = K.flatten(y_pred)
    targets = K.flatten(y_true)
    
    intersection = K.sum(K.dot(targets, inputs))
    dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    return 1 - dice

In [48]:
def DiceLoss(y_true, y_pred, smooth=1e-6):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    
    inputs = K.flatten(y_pred)
    targets = K.flatten(y_true)

    # Ajouter une dimension supplémentaire
    targets = K.expand_dims(targets, axis=-1)
    inputs = K.expand_dims(inputs, axis=-1)
    
    intersection = K.sum(targets * inputs)
    return (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)


In [49]:
y_true = np.array([[0, 1, 1],
                    [1, 0, 1],
                    [0, 0, 1]])

y_pred = np.array([[0.2, 0.8, 0.6],
                    [0.7, 0.3, 0.5],
                    [0.1, 0.4, 0.9]])

y_pred = np.array([[0, 1, 1],
                    [1, 0, 1],
                    [0, 0, 1]])

# Tester la fonction DiceLoss
loss = DiceLoss(y_true, y_pred)
print("Dice Loss:", loss)

Dice Loss: tf.Tensor(1.0, shape=(), dtype=float32)
